In [1]:
import scrapy
import requests
from bs4 import BeautifulSoup
import datetime
from textblob import TextBlob
import pandas as pd
import json
import numpy as np
import re

In [2]:
# [art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth]

In [3]:
class BigScraper:
    cols = ['art_content', 'art_content_html', 'art_published_datetime', 'art_lang', 'art_title',
            'art_url', 'src_name', 'src_type', 'src_url', 'art_img', 'art_auth', 'art_tag']

    def __init__(self):
        self.df = pd.DataFrame(columns=BigScraper.cols)

    def add_row(self, row_scrap):
        if type(row_scrap) == list:
            self.df.loc[len(self.df)] = row_scrap
        elif type(row_scrap) == dict:
            self.df = self.df.append(row_scrap, ignore_index=True)

    def get_base_url(url):
        for val in re.finditer("(\w)+://[^/]+/", url):
            return val.group(0)

    def scrap_changethework(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        art_html = soup.find('div', {'style': 'text-align: justify;'})
        art_content = art_html.get_text().strip().replace('\xa0', '')
        if soup.find('meta', {'property': 'article:modified_time'})['content'] == None:
            if soup.find('meta', {'property': 'article:published_time'})['content'] == None:
                art_extract_datetime = datetime.date.today()
            else:
                art_extract_datetime = soup.find(
                    'meta', {'property': 'article:published_time'})['content']
                art_extract_datetime = datetime.datetime.strptime(
                    art_extract_datetime, '%Y-%m-%dT%H:%M:%S%z').date()
        else:
            art_extract_datetime = soup.find(
                'meta', {'property': 'article:modified_time'})['content']
            art_extract_datetime = datetime.datetime.strptime(
                art_extract_datetime, '%Y-%m-%dT%H:%M:%S%z').date()
        art_lang = TextBlob(art_content).detect_language()
        art_title = soup.find('meta', {'property': 'og:title'})['content']
        art_url = soup.find('meta', {'property': 'og:url'})['content']
        src_name = soup.find('meta', {'property': 'og:site_name'})['content']
        src_type = 'xpath_source'
        src_url = BigScraper.get_base_url(art_url)
        src_img = soup.find('meta', {'property': 'og:image'})['content']
        art_auth = [el.get_text().strip() for el in soup.find_all(
            'span', class_='elementor-post-author')]
        art_tag = np.nan
        return [art_content, art_html, art_extract_datetime, art_lang, art_title, art_url, src_name, src_type, src_url, src_img, art_auth, art_tag]

    def scrap_fncrr(url):
        '''Documentation
        Parameters:
            url: url of the scraped page
        Out:
            row: dict of values
        '''
        response = requests.get(url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        # content, content_html
        try:
            content = html_soup.find("div", {'class': "contenu_c"})
            content_html = content
            content = content.text
        except:
            content_html = np.nan
            content = np.nan
        # date
        if html_soup.find("time", {'class': "updated"}) != None:
            date = html_soup.find("time", {'class': "updated"})
        else:
            date = html_soup.find("time", {'class': 'entry-date published'})
        try:
            date = date['datetime']
            date = datetime.datetime.strptime(
                date, '%Y-%m-%dT%H:%M:%S%z').date()
        except:
            # if no date is specified, put scraping date
            date = datetime.date.today()
        # tag, title
        presentation = html_soup.find("div", {'class': "prensentation"})
        tag = np.nan  # tags are not always interesting
        title = presentation.find("h1")
        title = title.text
        # Remplissage du dataframe
        row = {'art_content': content,
               'art_content_html': content_html,
               'art_published_datetime': date,
               'art_lang': 'fr',
               'art_title': title,
               'art_url': url,
               'src_name': 'fnccr',
               'src_type': 'xpath_source',
               'src_url': BigScraper.get_base_url(url),
               'art_img': np.nan,  # No images
               'art_auth': np.nan,  # No author specified
               'art_tag': tag}
        return row

    def scrap_sabbar(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        # Récupération du contenu de la page web (les paragraphes, avec et sans les balises html)
        art_content_html = soup.find('div', class_="entry-content")
        art_content = art_content_html.get_text().replace("\xa0", "").strip()
        # Extraction de la date de l'article
        art_extract_datetime = json.loads(soup.find(
            'script', class_='yoast-schema-graph yoast-schema-graph--main').get_text())['@graph'][1]['dateModified']
        art_extract_datetime = datetime.datetime.strptime(
            art_extract_datetime, '%Y-%m-%dT%H:%M:%S%z').date()
        # Langue de l'article
        art_lang = soup.find('meta', property="og:locale").get('content')
        # Titre
        art_title = soup.find('meta', property="og:title").get('content')
        # Url
        art_url = soup.find('link', rel='canonical').get('href')
        # Nom de la source
        src_name = "Sabbar"
        # Type de la source
        src_type = "xpath_source"
        # url source
        src_url = BigScraper.get_base_url(art_url)
        # Image(s)
        src_img = np.nan
        # Auteur de l'article
        art_auth = np.nan
        # Tag de l'auteur
        art_tag = np.nan

        return [art_content, art_content_html, art_extract_datetime, art_lang, art_title, art_url, src_name, src_type, src_url, src_img, art_auth, art_tag]

    def scrap_theinnovation(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        art_content_html = soup.find("div", {"class": "entry-content"})
        art_content = art_content_html.text.replace('\xa0', '')
        if soup.find("meta", {"property": "article:modified_time"}) != None:
            art_extract_datetime = soup.find(
                "meta", {"property": "article:modified_time"})["content"]
            art_extract_datetime = datetime.datetime.strptime(
                art_extract_datetime, '%Y-%m-%dT%H:%M:%S%z').date()
        elif soup.find("meta", {"property": "article:published_time"}) != None:
            art_extract_datetime = soup.find(
                "meta", {"property": "article:published_time"})["content"]
            art_extract_datetime = datetime.datetime.strptime(
                art_extract_datetime, '%Y-%m-%dT%H:%M:%S%z').date()
        else:
            art_extract_datetime = datetime.date.today()
        art_lang = soup.find("meta", {"property": "og:locale"})["content"]
        art_title = soup.find("meta", {"property": "og:title"})["content"]
        art_url = soup.find("meta", {"property": "og:url"})["content"]
        src_name = soup.find("meta", {"property": "og:site_name"})["content"]
        src_type = "xpath_source"  # default value
        src_url = BigScraper.get_base_url(art_url)
        src_img = soup.find("meta", {"property": "og:image"})["content"]
        art_auth = soup.find("a", {"rel": "author"}).text
        art_tag = soup.find("meta", {"name": "keywords"})["content"].split(',')
        return [art_content, art_content_html, art_extract_datetime, art_lang, art_title, art_url,
                src_name, src_type, src_url, src_img, art_auth, art_tag]

    def scrap_lemondeinformatique(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        art_content_html = soup.find('div', class_='article-body')
        art_content = art_content_html.get_text().replace('\xa0', '').strip()
        if soup.find("meta", {"itemprop": "datePublished"}) is not None:
            art_extract_datetime = soup.find(
                "meta", {"itemprop": "datePublished"})['content']
            art_extract_datetime = datetime.datetime.strptime(
                art_extract_datetime, '%Y-%m-%dT%H:%M:%S%z').date()
        else:
            art_extract_datetime = datetime.date.today()
        art_langue = TextBlob(get_Title(art_content)).detect_language()
        art_title = soup.find("meta", {"property": "og:title"})["content"]
        art_url = soup.find("meta", {"property": "og:url"})["content"]
        src_name = soup.find("meta", {"property": "og:site_name"})["content"]
        src_type = 'xpath_source'
        src_url = BigScraper.get_base_url(art_url)
        src_img = soup.find("meta", {"property": "og:image"})["content"]
        art_auth = soup.find(
            "div", class_="author-infos").find("b", {"itemprop": "name"}).get_text()
        art_tag = [el.get_text()
                   for el in soup.find_all("a", {"rel": "category tag"})]
        return [art_content, art_content_html, art_extract_datetime, art_lang, art_title, art_url, src_name, src_type, src_url, src_img, art_auth, art_tag]

    def scrap_riskinsight(url):
        """
        This function add different information of the webpage/article to the dataFrame
        """
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        art_content_html = soup.find('article')
        art_content = art_content_html.get_text()
        src_type = 'xpath_source'
        art_url = soup.find('meta', {'property': 'og:url'})['content']
        src_url = BigScraper.get_base_url(art_url)
        src_name = soup.find('meta', {'property': 'og:site_name'})['content']
        if soup.find("meta", {"name": "twitter:data1"}) is not None:
            art_auth = soup.find("meta", {"name": "twitter:data1"})['content']
        else:
            art_auth = 'no_data'
        if soup.find("meta", {"property": "article:modified_time"}) is not None:
            date = soup.find("meta", {"property": "article:modified_time"})[
                "content"]
            art_published_datetime = datetime.datetime.strptime(
                date, '%Y-%m-%dT%H:%M:%S%z').date()
        elif soup.find("meta", {"property": "article:published_time"}) is not None:
            date = soup.find("meta", {"property": "article:published_time"})[
                "content"]
            art_published_datetime = datetime.datetime.strptime(
                date, '%Y-%m-%dT%H:%M:%S%z').date()
        else:
            art_published_datetime = datetime.date.today()
        art_title = soup.title.get_text()
        art_lang = TextBlob(art_content).detect_language()
        if soup.find_all("a", {'class': "tag--link"}) is not None:
            art_tag = [el.get_text()
                       for el in soup.find_all("a", {'class': "tag--link"})]
        else:
            art_tag = 'no_data'
        if soup.find("meta", {"property": "og:image"}) is not None:
            art_img = soup.find(
                "meta", {"property": "og:image"})["content"]
        else:
            art_img = 'no_data'
        return [art_content, art_content_html, art_published_datetime, art_lang, art_title, art_url, src_name, src_type, src_url, art_img, art_auth, art_tag]

    def scrap_lebigdata(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        art_content_html = soup.find("article")
        art_content = art_content_html.get_text().replace('\xa0', '')
        if soup.find('meta', property='article:modified_time') is not None:
            art_published_datetime = soup.find(
                'meta', property='article:modified_time').get('content')
            art_published_datetime = datetime.datetime.strptime(
                art_published_datetime, '%Y-%m-%dT%H:%M:%S%z').date()
        elif soup.find('meta', property='article:published_time') is not None:
            art_published_datetime = soup.find(
                'meta', property='article:published_time').get('content')
            art_published_datetime = datetime.datetime.strptime(
                art_published_datetime, '%Y-%m-%dT%H:%M:%S%z').date()
        else:
            art_published_datetime = datetime.date.today()
        art_lang = soup.find('meta', property='og:locale').get('content')
        art_title = soup.find('meta', property="og:title").get('content')
        art_url = soup.find('meta', property="og:url").get('content')
        src_name = soup.find('meta', property="og:site_name").get('content')
        src_type = "xpath_source"
        src_url = BigScraper.get_base_url(art_url)
        art_img = soup.find('meta', property="og:image").get('content')
        art_auth = soup.find(
            'meta', attrs={'name': "twitter:data1"}).get('content')
        art_tag = np.nan
        return [art_content, art_content_html, art_published_datetime, art_lang, art_title, art_url, src_name, src_type, src_url, art_img, art_auth, art_tag]

    def assign_scraper(url):
        if 'https://changethework.com/' in url:
            return BigScraper.scrap_changethework(url)
        elif 'https://www.fnccr.asso.fr/article/' in url:
            return BigScraper.scrap_fncrr(url)
        elif 'http://sabbar.fr/' in url:
            return BigScraper.scrap_sabbar(url)
        elif 'https://www.theinnovation.eu/' in url:
            return BigScraper.scrap_theinnovation(url)
        elif 'https://www.lemondeinformatique.fr/' in url:
            return BigScraper.scrap_lemondeinformatique(url)
        elif 'https://www.riskinsight-wavestone.com/' in url:
            return BigScraper.scrap_riskinsight(url)
        elif 'https://www.lebigdata.fr/' in url:
            return BigScraper.scrap_lebigdata(url)
        return None

    def scrap(self, url):
        row = BigScraper.assign_scraper(url)
        self.add_row(row)
        return row

In [4]:
#     cols = ['art_content', 'art_content_html', 'art_published_datetime', 'art_lang', 'art_title',
#             'art_url', 'src_name', 'src_type', 'src_url', 'art_img', 'art_auth', 'art_tag']

In [70]:
def scrap_cadre_dirigeant_mag(url):
    art_content_html = soup.find('div', class_ = 'td-post-content').get_text()

In [96]:
url = "https://www.fnccr.asso.fr/article/big-data-territorial-publication-de-letude-de-la-fnccr/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [72]:
# ' \n'.join(text for text in soup.find_all(text=True) if text.parent.name in ['p', 'li', 'h1', 'h2', 'h3', 'ul', 'strong'])

In [163]:
url = 'https://www.riskinsight-wavestone.com/2019/08/detecter-incidents-machine-learning/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
for i in soup.find('article').contents:
    print(i.name)

h1
p
div
p
p
p
h2
p
ul
p
p
h2
p
p
h3
p
p
ul
p
p
h3
p
p
p
h3
p
ul
p
p
p
p
h3
p
ul
p
p
h3
p
p
ul
p
p
p
h2
p
p
p
p
table
p
p
p
footer


In [214]:
get_content("https://www.lebigdata.fr/base-de-donnees")[1]

'Les bases de données informatiques sont utilisées dans un grand nombre d’entreprises pour stocker, organiser et analyser les données. Découvrez tout ce que vous devez savoir à ce sujet : qu’est-ce qu’une base de données, à quoi sert-elle, comment fonctionne-t-elle, quelles sont les différentes catégories, et quelles sont les meilleures.Une base de données (que nous nommerons BDD par commodité) est une collection d’informations organisées afin d’être facilement consultables, gérables et mises à jour. Au sein d’une database, les données sont organisées en lignes, colonnes et tableaux. Elles sont indexées afin de pouvoir facilement trouver les informations recherchées à l’aide d’un logiciel informatique. Chaque fois que de nouvelles informations sont ajoutées, les données sont mises à jour, et éventuellement supprimées.Qu’est-ce qu’une base de données ? DéfinitionElles se chargent elles-mêmes de créer, de mettre à jour ou de supprimer des données. Elleseffectuent également des recherches

In [208]:
BG.df['art_content'][5].replace('\xa0', '')

"Base de données : qu’est-ce que c’est ? Définition et présentationBastien L\n24 janvier 2019\nDossiers\nCommentairesLes bases de données informatiques sont utilisées dans un grand nombre d’entreprises pour stocker, organiser et analyser les données. Découvrez tout ce que vous devez savoir à ce sujet : qu’est-ce qu’une base de données, à quoi sert-elle, comment fonctionne-t-elle, quelles sont les différentes catégories, et quelles sont les meilleures.Une base de données (que nous nommerons BDD par commodité) est une collection d’informations organisées afin d’être facilement consultables, gérables et mises à jour. Au sein d’une database, les données sont organisées en lignes, colonnes et tableaux. Elles sont indexées afin de pouvoir facilement trouver les informations recherchées à l’aide d’un logiciel informatique. Chaque fois que de nouvelles informations sont ajoutées, les données sont mises à jour, et éventuellement supprimées.Qu’est-ce qu’une base de données ? DéfinitionElles se c

In [115]:
BG.df['art_content'][2].replace('\xa0', '') == get_content("http://sabbar.fr/management/le-management-strategique-et-le-management-operationnel/#:~:text=Le%20management%20op%C3%A9rationnel%20correspond%20aux,pour%20atteindre%20les%20objectifs%20fix%C3%A9s.")[1]

False

In [196]:
BG = BigScraper()
row = BG.scrap('https://changethework.com/chatbot-rh-recrutement/')

In [197]:
BG.df

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,art_img,art_auth,art_tag
0,"Les chatbots, ou agents relationnels, sont dep...","[\n, [[Les , <a href=""https://changethework.co...",2017-11-28,fr,Chatbot RH : l'assistant fun du recrutement,https://changethework.com/chatbot-rh-recrutement/,Change the work,xpath_source,https://changethework.com/,https://changethework.com/wp-content/uploads/2...,"[Léo Bernard, Aurélien Leleux]",NaN


In [198]:
row = BG.scrap("https://www.fnccr.asso.fr/article/big-data-territorial-publication-de-letude-de-la-fnccr/")

In [199]:
BG.df

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,art_img,art_auth,art_tag
0,"Les chatbots, ou agents relationnels, sont dep...","[\n, [[Les , <a href=""https://changethework.co...",2017-11-28,fr,Chatbot RH : l'assistant fun du recrutement,https://changethework.com/chatbot-rh-recrutement/,Change the work,xpath_source,https://changethework.com/,https://changethework.com/wp-content/uploads/2...,"[Léo Bernard, Aurélien Leleux]",NaN
1,Si les regards se tournent souvent vers les gr...,[[Si les regards se tournent souvent vers les ...,2017-02-20,fr,"""Big data territorial"" : Publication de l'étud...",https://www.fnccr.asso.fr/article/big-data-ter...,fnccr,xpath_source,https://www.fnccr.asso.fr/,NaN,NaN,NaN


In [200]:
row = BG.scrap('http://sabbar.fr/management/le-management-strategique-et-le-management-operationnel/#:~:text=Le%20management%20op%C3%A9rationnel%20correspond%20aux,pour%20atteindre%20les%20objectifs%20fix%C3%A9s.')

In [201]:
BG.df

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,art_img,art_auth,art_tag
0,"Les chatbots, ou agents relationnels, sont dep...","[\n, [[Les , <a href=""https://changethework.co...",2017-11-28,fr,Chatbot RH : l'assistant fun du recrutement,https://changethework.com/chatbot-rh-recrutement/,Change the work,xpath_source,https://changethework.com/,https://changethework.com/wp-content/uploads/2...,"[Léo Bernard, Aurélien Leleux]",NaN
1,Si les regards se tournent souvent vers les gr...,[[Si les regards se tournent souvent vers les ...,2017-02-20,fr,"""Big data territorial"" : Publication de l'étud...",https://www.fnccr.asso.fr/article/big-data-ter...,fnccr,xpath_source,https://www.fnccr.asso.fr/,NaN,NaN,NaN
2,Qu'est-ce que le management ?\n\n\nLe manageme...,"[\n, [\n, [ ], \n], \n, [\n, [<span style=""fon...",2014-02-02,fr_FR,Le management stratégique et le management opé...,http://sabbar.fr/management/le-management-stra...,Sabbar,xpath_source,http://sabbar.fr/,NaN,NaN,NaN


In [202]:
row = BG.scrap('https://www.theinnovation.eu/comment-tuer-linnovation-avec-lanalyse-financiere/45')

In [203]:
BG.df

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,art_img,art_auth,art_tag
0,"Les chatbots, ou agents relationnels, sont dep...","[\n, [[Les , <a href=""https://changethework.co...",2017-11-28,fr,Chatbot RH : l'assistant fun du recrutement,https://changethework.com/chatbot-rh-recrutement/,Change the work,xpath_source,https://changethework.com/,https://changethework.com/wp-content/uploads/2...,"[Léo Bernard, Aurélien Leleux]",NaN
1,Si les regards se tournent souvent vers les gr...,[[Si les regards se tournent souvent vers les ...,2017-02-20,fr,"""Big data territorial"" : Publication de l'étud...",https://www.fnccr.asso.fr/article/big-data-ter...,fnccr,xpath_source,https://www.fnccr.asso.fr/,NaN,NaN,NaN
2,Qu'est-ce que le management ?\n\n\nLe manageme...,"[\n, [\n, [ ], \n], \n, [\n, [<span style=""fon...",2014-02-02,fr_FR,Le management stratégique et le management opé...,http://sabbar.fr/management/le-management-stra...,Sabbar,xpath_source,http://sabbar.fr/,NaN,NaN,NaN
3,\nFiche de lecture\nInnovation killers : how f...,"[\n, [[<strong>Fiche de lecture</strong>]], \n...",2010-05-14,fr_FR,Comment tuer l’innovation avec l’analyse finan...,https://www.theinnovation.eu/comment-tuer-linn...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[analyse financière, cash flow, discovery-driv..."


In [204]:
row = BG.scrap("https://www.riskinsight-wavestone.com/2019/08/detecter-incidents-machine-learning/")

In [205]:
BG.df

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,art_img,art_auth,art_tag
0,"Les chatbots, ou agents relationnels, sont dep...","[\n, [[Les , <a href=""https://changethework.co...",2017-11-28,fr,Chatbot RH : l'assistant fun du recrutement,https://changethework.com/chatbot-rh-recrutement/,Change the work,xpath_source,https://changethework.com/,https://changethework.com/wp-content/uploads/2...,"[Léo Bernard, Aurélien Leleux]",NaN
1,Si les regards se tournent souvent vers les gr...,[[Si les regards se tournent souvent vers les ...,2017-02-20,fr,"""Big data territorial"" : Publication de l'étud...",https://www.fnccr.asso.fr/article/big-data-ter...,fnccr,xpath_source,https://www.fnccr.asso.fr/,NaN,NaN,NaN
2,Qu'est-ce que le management ?\n\n\nLe manageme...,"[\n, [\n, [ ], \n], \n, [\n, [<span style=""fon...",2014-02-02,fr_FR,Le management stratégique et le management opé...,http://sabbar.fr/management/le-management-stra...,Sabbar,xpath_source,http://sabbar.fr/,NaN,NaN,NaN
3,\nFiche de lecture\nInnovation killers : how f...,"[\n, [[<strong>Fiche de lecture</strong>]], \n...",2010-05-14,fr_FR,Comment tuer l’innovation avec l’analyse finan...,https://www.theinnovation.eu/comment-tuer-linn...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[analyse financière, cash flow, discovery-driv..."
4,Détecter des incidents cyber par Machine Learn...,[[Détecter des incidents cyber par Machine Lea...,2019-12-31,fr,Détecter des incidents cyber par Machine Learn...,https://www.riskinsight-wavestone.com/2019/08/...,RiskInsight,xpath_source,https://www.riskinsight-wavestone.com/,https://www.riskinsight-wavestone.com/wp-conte...,Hugo MORET,"[#détection, #Intelligence Artificielle, #Mach..."


In [206]:
row = BG.scrap("https://www.lebigdata.fr/base-de-donnees")

In [207]:
BG.df

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,art_img,art_auth,art_tag
0,"Les chatbots, ou agents relationnels, sont dep...","[\n, [[Les , <a href=""https://changethework.co...",2017-11-28,fr,Chatbot RH : l'assistant fun du recrutement,https://changethework.com/chatbot-rh-recrutement/,Change the work,xpath_source,https://changethework.com/,https://changethework.com/wp-content/uploads/2...,"[Léo Bernard, Aurélien Leleux]",NaN
1,Si les regards se tournent souvent vers les gr...,[[Si les regards se tournent souvent vers les ...,2017-02-20,fr,"""Big data territorial"" : Publication de l'étud...",https://www.fnccr.asso.fr/article/big-data-ter...,fnccr,xpath_source,https://www.fnccr.asso.fr/,NaN,NaN,NaN
2,Qu'est-ce que le management ?\n\n\nLe manageme...,"[\n, [\n, [ ], \n], \n, [\n, [<span style=""fon...",2014-02-02,fr_FR,Le management stratégique et le management opé...,http://sabbar.fr/management/le-management-stra...,Sabbar,xpath_source,http://sabbar.fr/,NaN,NaN,NaN
3,\nFiche de lecture\nInnovation killers : how f...,"[\n, [[<strong>Fiche de lecture</strong>]], \n...",2010-05-14,fr_FR,Comment tuer l’innovation avec l’analyse finan...,https://www.theinnovation.eu/comment-tuer-linn...,TheInnovation,xpath_source,https://www.theinnovation.eu/,https://s0.wp.com/i/blank.jpg,Guy Parmentier,"[analyse financière, cash flow, discovery-driv..."
4,Détecter des incidents cyber par Machine Learn...,[[Détecter des incidents cyber par Machine Lea...,2019-12-31,fr,Détecter des incidents cyber par Machine Learn...,https://www.riskinsight-wavestone.com/2019/08/...,RiskInsight,xpath_source,https://www.riskinsight-wavestone.com/,https://www.riskinsight-wavestone.com/wp-conte...,Hugo MORET,"[#détection, #Intelligence Artificielle, #Mach..."
5,Base de données : qu’est-ce que c’est ? Défini...,"[[[]], [[<span itemprop=""name"">Base de données...",2019-01-24,fr_FR,Base de données : qu’est-ce que c’est ? Défini...,https://www.lebigdata.fr/base-de-donnees,LeBigData.fr,xpath_source,https://www.lebigdata.fr/,https://www.lebigdata.fr/wp-content/uploads/20...,Bastien L,NaN
